## Questão 15

#### A distributing company has two major customers and three supply sources. The corresponding unit from each supply center to each customer is given in the following table, together with the total customer requirements and supply availabilities.


| Supply Center         | 1   | 2   | Available Supplies |
|-----------------------|-----|-----|--------------------|
| 1                     | -1  | 3   | 300                |
| 2                     | 1   | 6   | 400                |
| 3                     | 1   | 5   | 900                |
| Customer requirements | 800 | 500 |                    |

#### Note that Customer 1 has strong preferences for Supplier 1 AND will be willing not only to absorb all the transportation costs but also to pay a premium price of $1 per unit of product coming from Supplier 1.


<b>a) The top management of the distributing company feels it is obvious that Supply Center 1 should send all its
available products to Customer 1. Is this necessarily so? (Hint. Obtain the least-cost solution to the problem.
Explore whether alternative optimal solutions exist where not all the 300 units available in Supply Center 1 are
assigned to Customer 1.)<b>


<b>b) Assume Customer 2 is located in an area where all shipments will be subject to taxes defined as a percentage of
the unit cost of a product. Will this tax affect the optimal solution of part (a)?<b>

<b>c) Ignore part (b). What will be the optimal solution to the original problem if Supply Center 1 increases its product
availability from 300 units to 400 units?<b>

### Resposta

In [40]:
using JuMP

In [41]:
using GLPK

### A)

Utilizando a dica, modelaremos o problema para encontrar a solução ótima. Para isso criamos as variáveis $x_{ij}$ onde i=1,2,3 e j=1,2 que indicam a quantidade de produtos que saem do centro i com direção ao cliente j. Temos as restrições de máximo de produto disponível nos centros e demanda de cada cliente. Desa forma, temos:

In [39]:
dem1 = 800
dem2 = 500

max1 = 300
max2 = 400
max3 = 900

custo11 = -1
custo12 = 3
custo21 = 1
custo22 = 6
custo31 = 1
custo32 = 5

5

min $\sum_{i=1}^{3}\sum_{j=1}^{2} x_{ij}\cdot custo_{ij}$

subject to

$x_{11} + x_{12} \leq max1$

$x_{21} + x_{22} \leq max2$

$x_{31} + x_{32} \leq max3$

$x_{11} + x_{21} + x_{31} \leq dem1$

$x_{12} + x_{22} + x_{32} \leq dem2$

$x_{ij} \geq 0 \qquad  (i=1,2,3)(j=1,2)$

$x_{ij}\quad integer \qquad(i=1,2,3)(j=1,2)$

In [46]:
# Modelo e Solver
model = Model(GLPK.Optimizer)

# Variaveis, canalizações (ou caixas) e tipo
@variable(model,x[1:3,1:2]>=0,Int)
Q = [custo11 custo12;custo21 custo22;custo31 custo32] # 3x2
max = [max1,max2,max3]
dem = [dem1,dem2]

# Restrições
@constraint(model,max_de_linha[i=1:3],sum(x[i,:]) <= max[i])
@constraint(model,demanda_linha[j=1:2],sum(x[:,j])==dem[j])

# Função objetivo
@objective(model,Min,sum(Q.*x))

print(model)

Min -x[1,1] + x[2,1] + x[3,1] + 3 x[1,2] + 6 x[2,2] + 5 x[3,2]
Subject to
 demanda_linha[1] : x[1,1] + x[2,1] + x[3,1] == 800.0
 demanda_linha[2] : x[1,2] + x[2,2] + x[3,2] == 500.0
 max_de_linha[1] : x[1,1] + x[1,2] <= 300.0
 max_de_linha[2] : x[2,1] + x[2,2] <= 400.0
 max_de_linha[3] : x[3,1] + x[3,2] <= 900.0
 x[1,1] >= 0.0
 x[2,1] >= 0.0
 x[3,1] >= 0.0
 x[1,2] >= 0.0
 x[2,2] >= 0.0
 x[3,2] >= 0.0
 x[1,1] integer
 x[2,1] integer
 x[3,1] integer
 x[1,2] integer
 x[2,2] integer
 x[3,2] integer


In [47]:
optimize!(model)

In [48]:
objective_value(model)

2700.0

In [49]:
value.(x)

3×2 Array{Float64,2}:
 300.0    0.0
 400.0    0.0
 100.0  500.0

Agora, fixamos que iremos enviar apenas 100 produtos do fornecedor 1 para o cliente 1:

In [50]:
# Modelo e Solver
model = Model(GLPK.Optimizer)

# Variaveis, canalizações (ou caixas) e tipo
@variable(model,x[1:3,1:2]>=0,Int)
Q = [-1 3;1 6;1 5] # 3x2
max = [max1,max2,max3]
dem = [dem1,dem2]

# Restrições
@constraint(model,max_de_linha[i=1:3],sum(x[i,:]) <= max[i])
@constraint(model,demanda_linha[j=1:2],sum(x[:,j])==dem[j])
@constraint(model,limite_x,x[1,1]==100)

# Função objetivo
@objective(model,Min,sum(Q.*x))

print(model)

Min -x[1,1] + x[2,1] + x[3,1] + 3 x[1,2] + 6 x[2,2] + 5 x[3,2]
Subject to
 demanda_linha[1] : x[1,1] + x[2,1] + x[3,1] == 800.0
 demanda_linha[2] : x[1,2] + x[2,2] + x[3,2] == 500.0
 limite_x : x[1,1] == 100.0
 max_de_linha[1] : x[1,1] + x[1,2] <= 300.0
 max_de_linha[2] : x[2,1] + x[2,2] <= 400.0
 max_de_linha[3] : x[3,1] + x[3,2] <= 900.0
 x[1,1] >= 0.0
 x[2,1] >= 0.0
 x[3,1] >= 0.0
 x[1,2] >= 0.0
 x[2,2] >= 0.0
 x[3,2] >= 0.0
 x[1,1] integer
 x[2,1] integer
 x[3,1] integer
 x[1,2] integer
 x[2,2] integer
 x[3,2] integer


In [51]:
optimize!(model)

In [52]:
objective_value(model)

2700.0

In [53]:
value.(x)

3×2 Array{Float64,2}:
 100.0  200.0
 400.0    0.0
 300.0  300.0

### B)

Supomos, nesse item, que o cliente 2 esteja em uma área tal que todos os envios estão sujeitos a uma taxa definida como porcentagem do custo de uma unidade. Definindo que, nesse local, temos uma taxa de 10%. Temos: 

In [54]:
dem1 = 800
dem2 = 500

max1 = 300
max2 = 400
max3 = 900

custo11 = -1
custo12 = 3.3
custo21 = 1
custo22 = 6.6
custo31 = 1
custo32 = 5.5

5.5

In [55]:
# Modelo e Solver
model = Model(GLPK.Optimizer)

# Variaveis, canalizações (ou caixas) e tipo
@variable(model,x[1:3,1:2]>=0,Int)
Q = [custo11 custo12;custo21 custo22;custo31 custo32] # 3x2
max = [max1,max2,max3]
dem = [dem1,dem2]

# Restrições
@constraint(model,max_de_linha[i=1:3],sum(x[i,:]) <= max[i])
@constraint(model,demanda_linha[j=1:2],sum(x[:,j])==dem[j])

# Função objetivo
@objective(model,Min,sum(Q.*x))

print(model)

Min -x[1,1] + x[2,1] + x[3,1] + 3.3 x[1,2] + 6.6 x[2,2] + 5.5 x[3,2]
Subject to
 demanda_linha[1] : x[1,1] + x[2,1] + x[3,1] == 800.0
 demanda_linha[2] : x[1,2] + x[2,2] + x[3,2] == 500.0
 max_de_linha[1] : x[1,1] + x[1,2] <= 300.0
 max_de_linha[2] : x[2,1] + x[2,2] <= 400.0
 max_de_linha[3] : x[3,1] + x[3,2] <= 900.0
 x[1,1] >= 0.0
 x[2,1] >= 0.0
 x[3,1] >= 0.0
 x[1,2] >= 0.0
 x[2,2] >= 0.0
 x[3,2] >= 0.0
 x[1,1] integer
 x[2,1] integer
 x[3,1] integer
 x[1,2] integer
 x[2,2] integer
 x[3,2] integer


In [56]:
optimize!(model)

In [57]:
objective_value(model)

2890.0

In [58]:
value.(x)

3×2 Array{Float64,2}:
   0.0  300.0
 400.0    0.0
 400.0  200.0

### C)

Supomos nesse item que o fornecedor 1 agora tem um máximo de 400 produtos disponíveis: 

In [59]:
dem1 = 800
dem2 = 500

max1 = 400
max2 = 400
max3 = 900

custo11 = -1
custo12 = 3
custo21 = 1
custo22 = 6
custo31 = 1
custo32 = 5

5

In [60]:
# Modelo e Solver
model = Model(GLPK.Optimizer)

# Variaveis, canalizações (ou caixas) e tipo
@variable(model,x[1:3,1:2]>=0,Int)
Q = [custo11 custo12;custo21 custo22;custo31 custo32] # 3x2
max = [max1,max2,max3]
dem = [dem1,dem2]

# Restrições
@constraint(model,max_de_linha[i=1:3],sum(x[i,:]) <= max[i])
@constraint(model,demanda_linha[j=1:2],sum(x[:,j])==dem[j])

# Função objetivo
@objective(model,Min,sum(Q.*x))

print(model)

Min -x[1,1] + x[2,1] + x[3,1] + 3 x[1,2] + 6 x[2,2] + 5 x[3,2]
Subject to
 demanda_linha[1] : x[1,1] + x[2,1] + x[3,1] == 800.0
 demanda_linha[2] : x[1,2] + x[2,2] + x[3,2] == 500.0
 max_de_linha[1] : x[1,1] + x[1,2] <= 400.0
 max_de_linha[2] : x[2,1] + x[2,2] <= 400.0
 max_de_linha[3] : x[3,1] + x[3,2] <= 900.0
 x[1,1] >= 0.0
 x[2,1] >= 0.0
 x[3,1] >= 0.0
 x[1,2] >= 0.0
 x[2,2] >= 0.0
 x[3,2] >= 0.0
 x[1,1] integer
 x[2,1] integer
 x[3,1] integer
 x[1,2] integer
 x[2,2] integer
 x[3,2] integer


In [61]:
optimize!(model)

In [62]:
objective_value(model)

2500.0

In [63]:
value.(x)

3×2 Array{Float64,2}:
 400.0    0.0
 400.0    0.0
   0.0  500.0